<a href="https://colab.research.google.com/github/faustian-echoes/uptitude_assignment/blob/main/uptitude_open_restaurants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#import modules
import pandas as pd
from datetime import datetime
import numpy as np

In [12]:
#Function which accepts file
def open_hotels(restaurant_file_path,date_time):

  #creating pandas dataframe using restaurant csv file 
  df = pd.read_csv(restaurant_file_path, names = ['restaurant_name','operation_time'])

  #drop rows with null values
  df = df.dropna(how='any',axis=0)

  #creating rows from columns by using '/' as separator using explode function
  df = (df.set_index(['restaurant_name']).apply(lambda x: x.str.split('/').explode()).reset_index()) 

  #removing extra spaces
  df['operation_time'] = df['operation_time'].str.strip()

  #splitting the operation_time column to capture start time, end time and days of operation value.
  #Storing them in a temporary df
  temp = df["operation_time"].str.rsplit(" ", n = 5, expand = True)
  df ['days'] = temp [0]
  df ['start_time'] = temp [1] +' ' + temp [2]
  df ['end_time'] = temp[4] + ' ' + temp[5]

  #Drop operaation time
  df = df.drop(['operation_time'], axis=1)

  #Splitting columns into rows based on ',' separator in days column.
  df_final = (df.set_index(['restaurant_name','start_time','end_time']).apply(lambda x: x.str.split(',').explode()).reset_index()) 

  #Splitting days column into 2 columns based on '-' separator and storing in temp df
  temp = df_final["days"].str. rsplit("-", n = 1, expand = True)

  #Creating start and end day columns from temp df
  df_final['start_day'] = temp[0]
  df_final['end_day'] = temp[1]

  #dropping days column as we have info available in start and end day columns
  df_final = df_final.drop(['days'], axis=1)

  #for columns where end day is null, end day is populated same as start day
  #e.g.- restaurants only operational on single day.
  df_final['end_day'].fillna(df_final['start_day'], inplace = True)

  #Removing extra spaces
  df_final['start_day'] = df_final['start_day'].str.strip()
  df_final['end_day'] = df_final['end_day'].str.strip()

  #Creating a dictionary with day and weekday numbers
  dict_days = {'Mon' : 0 , 'Tue' : 1 , 'Wed' : 2 , 'Thu' : 3 , 'Fri' : 4 , 'Sat' : 5 , 'Sun' : 6}

  #Replacing start and end day values by dictionay values so that it's easier to compare
  df_final = df_final.replace({"start_day": dict_days})
  df_final = df_final.replace({"end_day": dict_days})

  #cleaning the start and end time columns to represent in proper 24 hr time format.
  df_final['start_time'] = np.where (df_final['start_time'].str.contains(':'),df_final["start_time"],df_final["start_time"].str.rsplit(" ", n = 1, expand = True)[0] + ':00 ' + df_final["start_time"].str.rsplit(" ", n = 1, expand = True)[1])
  df_final['end_time'] = np.where (df_final['end_time'].str.contains(':'),df_final["end_time"],df_final["end_time"].str.rsplit(" ", n = 1, expand = True)[0] + ':00 ' + df_final["end_time"].str.rsplit(" ", n = 1, expand = True)[1])

  df_final['start_time'] = pd.to_datetime(df_final['start_time'])
  df_final['end_time'] = pd.to_datetime(df_final['end_time'])

  df_final['start_time'] = df_final['start_time'].dt.strftime('%H:%M')
  df_final['end_time'] = df_final['end_time'].dt.strftime('%H:%M')

  #Replacing ':' from 24 hr time format values to give a proper integer value
  df_final['start_time'] = df_final['start_time'].str.replace(':','')
  df_final['end_time'] = df_final['end_time'].str.replace(':','')

  #This special condition is for restaurants that operate beyond midnight.
  #For our derivations, adding 2400 hrs to their end time if end _time < start_time
  df_final['end_time'] = np.where (df_final['end_time'] < df_final['start_time'],
  df_final['end_time'].astype(int) + 2400, df_final['end_time'])


  #getting time and weekday information from input datetime value
  date_time_string = str(date_time)[11:16]

  weekday = date_time.weekday()
  #print(weekday)

  date_time_string = date_time_string.replace(':','')
  #print(date_time_string)

  #Finally storing the open restaurant names in a separate df
  #checking if weekday from input datetime values lies between start and end day value of a restaurant operation
  #also checking the time from datetime value lies between restaurants opening and closing time for that particular weekday.
  df_final_open = df_final[(df_final['start_day'] <= weekday) & (df_final['end_day'] >= weekday) & (df_final['start_time'] <= date_time_string) & (df_final['end_time'].astype(str) >= date_time_string)]

  #printing list of open restaurants.
  print(df_final_open['restaurant_name'].to_string(index=False))
